In [1]:
import pandas as pd
df = pd.read_csv('./boolocally_poi/선별 테마와 비교-표 1.csv')
df.head()

,POI,커스텀맵 주제,커스텀맵 주제.1,커스텀맵 주제.2,운영팀 테마3,태그 추출 태그 top3,태그 추출 상세,태그 관련 하위어 점수
0,1723,"고요한 쉼의 공간, 소박한 마당이 있는 전주 한옥 카페",한옥,음식점,카페,NaN,NaN,NaN
1,외할머니솜씨,"고요한 쉼의 공간, 소박한 마당이 있는 전주 한옥 카페",한옥,음식점,카페,NaN,NaN,NaN
2,목련을부탁해,"고요한 쉼의 공간, 소박한 마당이 있는 전주 한옥 카페",한옥,음식점,카페,NaN,NaN,NaN
3,명천재,"고요한 쉼의 공간, 소박한 마당이 있는 전주 한옥 카페",한옥,음식점,카페,NaN,NaN,NaN
4,차경,"고요한 쉼의 공간, 소박한 마당이 있는 전주 한옥 카페",한옥,음식점,카페,NaN,NaN,NaN


In [4]:
len(df)

174

In [2]:
poi_names = df['POI'].tolist()
poi_names[:2]

['1723', '외할머니솜씨']

In [3]:
from dbConn.postgresConn import get_select
# 네이버네임

no_result = []
a, b, c = [], [], []
for poi_name in poi_names:
    query = f"""select analyzed_json from scrapping.scrapping.instagram_location where naver_name like '%{poi_name}%' and analyzed_json is not null;"""
    result = get_select(query)
    best_result = {'num_of_documents': -1}
    for (json_data, ) in result:
        if json_data and json_data['num_of_documents'] > best_result['num_of_documents']:
            best_result = json_data
    if best_result['num_of_documents'] != -1:
        a.append(best_result['top_tags'])
        b.append(sorted(best_result['tags_score'].items(), key=lambda x: -x[1]))
        c.append(best_result['tags_hyponym'])
    else:
        a.append(None)
        b.append(None)
        c.append(None)

In [22]:
no_result

[]

In [5]:
len(a)

174

In [6]:
df['태그 추출 태그 top3'] = pd.Series(a)
df['태그 추출 상세'] = pd.Series(b)
df['태그 관련 하위어 점수'] = pd.Series(c)

In [7]:
df.head()

,POI,커스텀맵 주제,커스텀맵 주제.1,커스텀맵 주제.2,운영팀 테마3,태그 추출 태그 top3,태그 추출 상세,태그 관련 하위어 점수
0,1723,"고요한 쉼의 공간, 소박한 마당이 있는 전주 한옥 카페",한옥,음식점,카페,"[디저트, 카페, 맛집]","[(디저트, 29.452240731321222), (카페, 24.0831700240...","{'가족': {'가족': 0.39044788707327016}, '맛집': {'맛집..."
1,외할머니솜씨,"고요한 쉼의 공간, 소박한 마당이 있는 전주 한옥 카페",한옥,음식점,카페,"[맛집, 카페, 연인]","[(맛집, 29.24147610299455), (카페, 28.382306190403...","{'가족': {'가족': 0.523418189740747, '기념': 0.20310..."
2,목련을부탁해,"고요한 쉼의 공간, 소박한 마당이 있는 전주 한옥 카페",한옥,음식점,카페,"[디저트, 카페, 연인]","[(디저트, 27.792656593998927), (카페, 23.1400884090...","{'맛집': {'맛집': 1.3035088621353879, '먹방': 0.8450..."
3,명천재,"고요한 쉼의 공간, 소박한 마당이 있는 전주 한옥 카페",한옥,음식점,카페,None,None,None
4,차경,"고요한 쉼의 공간, 소박한 마당이 있는 전주 한옥 카페",한옥,음식점,카페,"[디저트, 카페, 사진]","[(디저트, 6.1306463716079005), (카페, 5.83110373842...","{'노을': {'풍경': 0.9464083380511845}, '등산': {'단풍'..."


In [8]:
df.to_csv('./boolocally_poi_tags.csv', index=False, encoding='utf-8')